In [0]:
# ------------------------------
# 
# This script does the following:
# 
#     1. Reads the bronze table '

# 
# --------------------------------------


import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, FloatType

catalog_name = 'ecommerce'


## Brands

In [0]:
df_bronze = spark.table(f'{catalog_name}.bronze.brz_brands')
display(df_bronze.limit(10))
# Columns that needs transformation:


In [0]:
df_silver = df_bronze.withColumn('brand_name',F.trim(F.col('brand_name')))\
              .withColumn("brand_code", F.regexp_replace(F.col("brand_code"), r'[^A-Za-z0-9]', ''))
display(df_silver.limit(10))


In [0]:
df_silver.select("category_code").distinct().show()
# Ununified categories codes

In [0]:
anomalies = {
    "GROCERY": "GRCY",
    "BOOKS": "BKS",
    "TOYS": "TOY"
}

df_silver = df_silver.replace(to_replace=anomalies, subset=["category_code"])

# Show results
df_silver.select("category_code").distinct().show()


In [0]:
# Writing

df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_brands")

## Category

In [0]:
df_bronze = spark.table(f"{catalog_name}.bronze.brz_category")
display(df_bronze.limit(5))